In [8]:
import os

os.environ['KERAS_BACKEND'] = "torch"

## Kickstarter Less Dataset (256k)


In [9]:
from train import AbaeModelConfiguration
from core.train import AbaeModelManager

config = AbaeModelConfiguration(
    corpus_file="../data/processed-dataset/full/64k.preprocessed.csv",
    model_name="abae.full.64k", aspect_size=16
)

manager = AbaeModelManager(config)

Pandas Apply:   0%|          | 0/80286 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/80286 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\abae.full.64k.embeddings.keras
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\abae.full.64k.embeddings.keras', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\abae.full.64k.embeddings.keras.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\abae.full.64k.embeddings.keras', 'datetime': '2024-12-21T19:34:10.904741', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}


In [10]:
inference_model = manager.prepare_evaluation_model()

D:\PycharmProjects\nlp-course-project\core\layer.py:126: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(WeightedAspectEmb, self).__init__(**kwargs)


In [11]:
import torch

#word_emb = inference_model.get_layer('word_embedding').weights[0]
word_emb = inference_model.get_layer('word_embedding').weights[0].value.data
#word_emb = torch.from_numpy(word_emb)

word_emb.shape

torch.Size([9004, 128])

In [12]:
aspect_embeddings = inference_model.get_layer('weighted_aspect_emb').W
vocab_inv = manager.embedding_model.model.wv.index_to_key

In [13]:
from core.evaluation import normalize_embedding_matrix

word_emb = normalize_embedding_matrix(word_emb)
aspect_embeddings = normalize_embedding_matrix(aspect_embeddings)

In [14]:
from core.evaluation import extract_top_k_words_of_aspect

aspects_top_k_words = [extract_top_k_words_of_aspect(a, word_emb, vocab_inv) for a in aspect_embeddings]


Given aspect most representative words are:
Word:  legendary (0.4351903200149536)
Word:  marvel (0.42146116495132446)
Word:  include (0.41859087347984314)
Word:  promos (0.40995022654533386)
Word:  universe (0.4064077138900757)
Word:  cards (0.3850122392177582)
Word:  expansions (0.3809184432029724)
Word:  unmatched (0.38064125180244446)
Word:  north (0.37879279255867004)
Word:  expansion (0.37542006373405457)
Word:  w/ (0.37347477674484253)
Word:  ✓ (0.36681437492370605)
Word:  <game_name> (0.36548852920532227)
Word:  promo (0.36144232749938965)
Word:  shrink (0.3540777862071991)
Word:  american (0.3519633412361145)
Word:  flight (0.3502519726753235)
Word:  pledge (0.34722787141799927)
Word:  hill (0.3468876779079437)
Word:  premium (0.346232533454895)
Word:  wars (0.34497418999671936)
Word:  exp (0.339688777923584)
Word:  cthulhu (0.33938536047935486)
Word:  wing (0.33663520216941833)
Word:  european (0.33345741033554077)

Given aspect most representative words are:
Word:  factor (0

![title](./../resources/umass.png)


### Dataset generation

In [18]:
import dataset

vocabulary = manager.embedding_model.model.wv.key_to_index
ds = dataset.PositiveNegativeCommentGeneratorDataset(
    vocabulary=vocabulary, csv_dataset_path="../data/processed-dataset/full/64k.preprocessed.csv", negative_size=15
)

Loading dataset from file: ../data/processed-dataset/full/64k.preprocessed.csv
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/80286 [00:00<?, ?it/s]

Max sequence length calculation in progress...
Max sequence length is:  206 . The limit is set to 256 tokens.
Padding sequences to length (256).


In [19]:
from core.evaluation import coherence

print(manager.embedding_model.model.wv.index_to_key[13])
# todo pass indexwords
for i in range(len(aspects_top_k_words)):
    i_th_aspect_top_k = aspects_top_k_words[i][0:10]
    word_indexes = list(map(lambda x: x[2], i_th_aspect_top_k))
    print(word_indexes)
    print(coherence(word_indexes, ds))

lot
[tensor(1099, device='cuda:0'), tensor(986, device='cuda:0'), tensor(152, device='cuda:0'), tensor(1749, device='cuda:0'), tensor(1247, device='cuda:0'), tensor(1082, device='cuda:0'), tensor(2420, device='cuda:0'), tensor(1468, device='cuda:0'), tensor(1991, device='cuda:0'), tensor(49, device='cuda:0')]


KeyboardInterrupt: 

4.4. Word2vec Coherence Score
[https://www.baeldung.com/cs/topic-modeling-coherence-score#4-word2vec-coherence-score